In [1]:
from data.models import *
import data.models as models
from peewee import *
import json
import pandas as pd

models.init_db('./data/data.db')
models.create_tables()

# Bus flow by Bus Stop

In [2]:
q = BusStop.select()
bus_stop = pd.DataFrame(list(q.dicts()))
bus_stop['route_pass'] = 0
bus_stop['shift_pass'] = 0
bus_stop.head()

,id,year,month,stop_code,parent_code,name_cht,name_por,loc_lat,loc_lon,parish,stats_zone,route_pass,shift_pass
0,1,2016,2,M1,M1,關閘總站,Portas do Cerco / Terminal,22.214503,113.548907,1.0,13.0,0,0
1,2,2016,2,M2,M2,東方明珠總站,Pérola Oriental / Terminal,22.210808,113.557108,1.0,20.0,0,0
2,3,2016,2,M3,M3,媽閣總站,Barra / Terminal,22.184697,113.528953,2.0,15.0,0,0
3,4,2016,2,M4,M4,菜園涌邊街總站,R. Marginal do Canal das Hortas / Terminal,22.213657,113.544863,1.0,13.0,0,0
4,5,2016,2,M5,M5,街總社區服務大樓,União G. das Associações dos Moradores,22.213356,113.544050,1.0,13.0,0,0


In [3]:
q = SubRouteBusStopRaw.select()
sub_bus_stop = pd.DataFrame(list(q.dicts()))
sub_bus_stop['route_pass'] = 0
sub_bus_stop['shift_pass'] = 0
sub_bus_stop.head()
# sub_bus_stop['raw_stop_code'].unique()

,id,year,month,raw_stop_code,loc_lat,loc_lon,parish,stats_zone,route_pass,shift_pass
0,1,2016,2,M26,22.208334,113.543538,1.0,17.0,0,0
1,2,2016,2,M10,22.210444,113.546125,1.0,13.0,0,0
2,3,2016,2,M27,22.210711,113.548525,1.0,23.0,0,0
3,4,2016,2,M29,22.208519,113.550256,1.0,23.0,0,0
4,5,2016,2,M36,22.206360,113.552942,1.0,8.0,0,0


In [4]:
q = SubRoutePoint.select()
sub_route_point = pd.DataFrame(list(q.dicts()))
sub_route_point.head()

,id,year,month,sub_route,seq,loc_lat,loc_lon,sub_route_stop,raw_stop_code,parish,stats_zone
0,1,2016,2,3,0,22.208334,113.543538,1.0,1.0,1.0,None
1,2,2016,2,3,1,22.208605,113.543447,NaN,NaN,1.0,None
2,3,2016,2,3,2,22.208979,113.543323,NaN,NaN,1.0,None
3,4,2016,2,3,3,22.209166,113.543261,NaN,NaN,1.0,None
4,5,2016,2,3,4,22.209691,113.543104,NaN,NaN,1.0,None


In [5]:
q = SubRoutePoint.select() \
    .where(SubRoutePoint.raw_stop_code != None)
sub_stop_mapping = pd.DataFrame(list(q.dicts())).loc[:, ['sub_route_stop', 'raw_stop_code']]
sub_stop_mapping.head()

,sub_route_stop,raw_stop_code
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [6]:
# bus_stop.groupby(['year', 'month']).count()
months = bus_stop[['year', 'month']].drop_duplicates()
months = list(zip(months['year'],months['month']))
print(months)

[(2016, 2), (2016, 4), (2016, 5), (2016, 7), (2016, 8), (2017, 2), (2017, 4), (2017, 5), (2017, 6), (2017, 8), (2017, 9), (2018, 1), (2018, 4), (2018, 5), (2018, 7), (2018, 8), (2018, 9), (2018, 10), (2018, 12), (2019, 2), (2019, 3), (2019, 4), (2019, 5), (2019, 6), (2019, 8), (2019, 9), (2019, 10)]


In [7]:
q = BusSubRoute.select()
sub_route = pd.DataFrame(list(q.dicts()))
sub_route['shift'] = 0
sub_route.head()

,id,year,month,route,sub_route_code,back_forth,name_cht,name_por,shift
0,1,2016,2,1,1,0,關閘 ⇀ 媽閣,Portas do Cerco ⇀ Barra,0
1,2,2016,2,1,1,1,媽閣 ⇀ 關閘,Barra ⇀ Portas do Cerco,0
2,3,2016,2,2,1A,0,筷子基 ⇀ 新口岸/科英布拉街,Fai Chi Kei ⇀ NAPE / Rua de Coimbra,0
3,4,2016,2,2,1A,1,新口岸/科英布拉街 ⇀ 筷子基,NAPE / Rua de Coimbra ⇀ Fai Chi Kei,0
4,5,2016,2,3,2,0,順景廣場 ⇀ 媽閣,P. Serenidade ⇀ Barra,0


In [8]:
q = SubRouteSchedule.select().where(SubRouteSchedule.day_type == 'weekdays')
sub_route_schedule = pd.DataFrame(list(q.dicts()))

In [9]:
# Get schedules
bus_flow = sub_route_schedule.groupby(['sub_route']).size()
for index, val in bus_flow.iteritems():
    sub_route.loc[index - 1, 'shift'] = val
    
sub_route[:10]

,id,year,month,route,sub_route_code,back_forth,name_cht,name_por,shift
0,1,2016,2,1,1,0,關閘 ⇀ 媽閣,Portas do Cerco ⇀ Barra,134
1,2,2016,2,1,1,1,媽閣 ⇀ 關閘,Barra ⇀ Portas do Cerco,137
2,3,2016,2,2,1A,0,筷子基 ⇀ 新口岸/科英布拉街,Fai Chi Kei ⇀ NAPE / Rua de Coimbra,125
3,4,2016,2,2,1A,1,新口岸/科英布拉街 ⇀ 筷子基,NAPE / Rua de Coimbra ⇀ Fai Chi Kei,125
4,5,2016,2,3,2,0,順景廣場 ⇀ 媽閣,P. Serenidade ⇀ Barra,127
5,6,2016,2,3,2,1,媽閣 ⇀ 順景廣場,Barra ⇀ P. Serenidade,124
6,7,2016,2,4,2A,0,東方明珠 ⇀ 亞馬喇前地,Pérola Oriental ⇀ Praça Ferreira Amaral,105
7,8,2016,2,4,2A,1,亞馬喇前地 ⇀ 東方明珠,Praça Ferreira Amaral ⇀ Pérola Oriental,104
8,9,2016,2,5,3,0,關閘 ⇀ 外港碼頭,Portas do Cerco ⇀ Terminal Marítimo,141
9,10,2016,2,5,3,1,外港碼頭 ⇀ 關閘,Terminal Marítimo ⇀ Portas do Cerco,144


In [10]:
q = SubRouteStop.select()
sub_route_stop = pd.DataFrame(list(q.dicts()))
sub_route_stop.head()

,id,year,month,sub_route,stop_seq,bus_stop
0,1,2016,2,3,1,22
1,2,2016,2,3,2,9
2,3,2016,2,3,3,23
3,4,2016,2,3,4,25
4,5,2016,2,3,5,30


In [11]:
for index, row in sub_route_stop.iterrows():
    bus_stop_idx = row['bus_stop'] - 1
    sub_route_idx = row['sub_route'] - 1
    sub_bus_stop_idx = sub_stop_mapping.loc[index, 'raw_stop_code'] - 1
    bus_stop.loc[bus_stop_idx, 'route_pass'] += 1
    bus_stop.loc[bus_stop_idx, 'shift_pass'] += sub_route.loc[sub_route_idx, 'shift']
    sub_bus_stop.loc[sub_bus_stop_idx, 'stop_code'] = bus_stop.loc[bus_stop_idx, 'stop_code']
    sub_bus_stop.loc[sub_bus_stop_idx, 'route_pass'] += 1
    sub_bus_stop.loc[sub_bus_stop_idx, 'shift_pass'] += sub_route.loc[sub_route_idx, 'shift']

In [12]:
sub_bus_stop.head()

,id,year,month,raw_stop_code,loc_lat,loc_lon,parish,stats_zone,route_pass,shift_pass,stop_code
0,1,2016,2,M26,22.208334,113.543538,1.0,17.0,8,1100,M26
1,2,2016,2,M10,22.210444,113.546125,1.0,13.0,18,1930,M10
2,3,2016,2,M27,22.210711,113.548525,1.0,23.0,7,688,M27
3,4,2016,2,M29,22.208519,113.550256,1.0,23.0,5,472,M29
4,5,2016,2,M36,22.206360,113.552942,1.0,8.0,10,1072,M36


In [13]:
res = dict()

for (year, month) in months:
    bus_stop_year = bus_stop['year'] == year
    bus_stop_month = bus_stop['month'] == month
    res[str(year) + '-' + str(month)] = json.loads(bus_stop[bus_stop_year & bus_stop_month].loc[:, ['stop_code', 'loc_lat', 'loc_lon', 'route_pass', 'shift_pass']].to_json(orient='records'))

with open('./stops.json', 'w') as f:
    f.write(json.dumps(res))


In [14]:
res = dict()

for (year, month) in months:
    bus_stop_year = sub_bus_stop['year'] == year
    bus_stop_month = sub_bus_stop['month'] == month
    res[str(year) + '-' + str(month)] = json.loads(sub_bus_stop[bus_stop_year & bus_stop_month].loc[:, ['stop_code', 'loc_lat', 'loc_lon', 'route_pass', 'shift_pass']].to_json(orient='records'))

with open('./sub_stops.json', 'w') as f:
    f.write(json.dumps(res))


In [15]:
import numpy

def getEquidistantPoints(p1, p2, parts):
    return zip(numpy.linspace(p1[0], p2[0], parts+1), numpy.linspace(p1[1], p2[1], parts+1))

In [16]:
res = dict()

for (year, month) in months[:1]:
    records = []
    sub_route_year = sub_route['year'] == year
    sub_route_month = sub_route['month'] == month
    routes = sub_route[sub_route_year & sub_route_month].sort_values(by='shift', ascending=False)
    for index, row in routes.iterrows():
        sub_route_id = row['id']
        q = SubRoutePoint.select().where(SubRoutePoint.sub_route == sub_route_id).where(SubRoutePoint.year == year).where(SubRoutePoint.month == month).order_by(SubRoutePoint.seq)
        q = [[x['loc_lon'], x['loc_lat']] for x in list(q.dicts())][:-1]
        a = [] 
        for idx, val in enumerate(q):
#             lval = q[idx - 1]
#             diff = max(abs(val[0]-lval[0]), abs(val[1]-lval[1]))
#             if (diff < 0.01 and diff > 0.0001):
#                 a += list(getEquidistantPoints(lval, val, diff * 10000))[1:]
#             else:
            a.append(val)
        records += [[x[0], x[1], row['shift'], row['id']] for x in a]
    res[str(year) + '-' + str(month)] = records

with open('./routes.json', 'w') as f:
    f.write(json.dumps(res))

In [17]:
q = SubRouteDistance.select()
sub_route_distance = pd.DataFrame(list(q.dicts()))
sub_route_distance.head()

,id,year,month,sub_route,seq,from_stop,to_stop,distance
0,1,2016,2,1,1,26,68,0.479107
1,2,2016,2,1,2,68,27,0.194265
2,3,2016,2,1,3,27,28,0.194153
3,4,2016,2,1,4,28,24,0.255378
4,5,2016,2,1,5,24,25,0.320268


In [18]:
# y = sub_route_distance['year'] == 2016
# m = sub_route_distance['month'] == 2
# r = sub_route_distance['sub_route'] == 1
# t['sub_route'] = sub_route.loc[t['from_stop'] - 1, ['sub_route_stop']]
# t.head()

sub_route_distance['sub_route_code'] = sub_route.loc[sub_route_distance['sub_route'] - 1, 'sub_route_code'].values
sub_route_distance['back_forth'] = sub_route.loc[sub_route_distance['sub_route'] - 1, 'back_forth'].values
sub_route_distance['name'] = sub_route.loc[sub_route_distance['sub_route'] - 1, 'name_cht'].values
sub_route_distance['fsc'] = sub_bus_stop.loc[sub_route_distance['from_stop'] - 1, 'stop_code'].values
sub_route_distance['tsc'] = sub_bus_stop.loc[sub_route_distance['to_stop'] - 1, 'stop_code'].values
sub_route_distance.head()

,id,year,month,sub_route,seq,from_stop,to_stop,distance,sub_route_code,back_forth,name,fsc,tsc
0,1,2016,2,1,1,26,68,0.479107,1,0,關閘 ⇀ 媽閣,M1,M225
1,2,2016,2,1,2,68,27,0.194265,1,0,關閘 ⇀ 媽閣,M225,M7
2,3,2016,2,1,3,27,28,0.194153,1,0,關閘 ⇀ 媽閣,M7,M229
3,4,2016,2,1,4,28,24,0.255378,1,0,關閘 ⇀ 媽閣,M229,M12
4,5,2016,2,1,5,24,25,0.320268,1,0,關閘 ⇀ 媽閣,M12,M31


In [19]:
res = dict()

for (year, month) in months:
    y = sub_route_distance['year'] == year
    m = sub_route_distance['month'] == month
    res[str(year) + '-' + str(month)] = json.loads(sub_route_distance[y & m].loc[:, ['name','distance', 'fsc', 'tsc', 'sub_route', 'sub_route_code', 'back_forth']].to_json(orient='records'))

with open('./distance.json', 'w') as f:
    f.write(json.dumps(res))